In [168]:
import psycopg2
import keys
import pandas as pd
import stockstats
from tapy import Indicators

print(2)

2


In [169]:
conn = psycopg2.connect(database="postgres", user=keys.user, password=keys.password, host=keys.host, port="5432")
cur = conn.cursor()
#cur.execute(f"select * from master_ticker_list limit 5;")
cur.execute(f"select * from historical_stock_data where ticker_id in (select id from master_ticker_list limit 100) and date >= '2021-01-20';")
data = cur.fetchall()
print(data)
conn.commit()
conn.close()


[(509259561, Decimal('10.010000228881836'), Decimal('10.420000076293945'), Decimal('10.5'), Decimal('9.6899995803833'), Decimal('338700.0'), '2021-01-20 00:00:00'), (509259561, Decimal('10.300000190734863'), Decimal('12.550000190734863'), Decimal('15.199999809265137'), Decimal('9.899999618530273'), Decimal('5942800.0'), '2021-01-21 00:00:00'), (509259561, Decimal('12.149999618530273'), Decimal('11.779999732971191'), Decimal('12.3100004196167'), Decimal('10.90999984741211'), Decimal('1096700.0'), '2021-01-22 00:00:00'), (509259561, Decimal('12.029999732971191'), Decimal('12.65999984741211'), Decimal('13.539999961853027'), Decimal('12.0'), Decimal('616100.0'), '2021-01-25 00:00:00'), (509259561, Decimal('12.890000343322754'), Decimal('13.09000015258789'), Decimal('13.449999809265137'), Decimal('12.319999694824219'), Decimal('398700.0'), '2021-01-26 00:00:00'), (509259561, Decimal('12.619999885559082'), Decimal('12.029999732971191'), Decimal('12.989999771118164'), Decimal('11.920000076293

In [170]:
#put in group arrays by ticker
grouped_data = []
arr = []
for item in data:
    if len(arr) == 0:
        arr.append(item)
    if arr[0][0] == item[0]:
        arr.append(item)
    if arr[0][0] != item[0]:
        grouped_data.append(arr)
        arr = []
        arr.append(item)
    if item == data[-1]:
        arr.append(item)
        grouped_data.append(arr)    


In [171]:
#make dfs from each group, run the rsi to see if optimal
#ticker_id, open, close, high, low, volume, date
tickers = []
aos = []
for tick in grouped_data:
    #one stock array filled with many arrays
    name = []
    _open = []
    close = []
    high = []
    low = []
    volume = []
    index = []
    for b in tick:
        name.append(b[0])
        _open.append(b[1])
        close.append(b[2])
        high.append(b[3])
        low.append(b[4])
        volume.append(b[5])
        index.append(b[6])
    data = {
        'ticker_id': name,
        'open': _open,
        'close': close,
        'high':high,
        'low': low,
        'volume': volume,
        'date': index,
    }
    cols = ['ticker_id', 'open', 'close', 'high', 'low', 'volume', 'date']
    
    df = pd.DataFrame (data, columns = cols)
    df.index = index
    stock = stockstats.StockDataFrame.retype(df)
    stock['rsi_12']
    stock.rename(columns={
        "high": "High",
        'low':'Low',
        'close': 'Close'
    }, inplace=True)

    stock = Indicators(stock)

    stock.awesome_oscillator(column_name='ao')
    
    stock = stock.df
    for c in stock['rsi_12'][-11:-1]:
        if c < 40 and c > 0:
            if stock['ticker_id'][0] not in tickers:
                tickers.append(stock['ticker_id'][0])
    #print(stock['ao'][-10:-1])      
    ao = stock['ao'][-11:-1]
    for n in range(len(ao)):
        if ao[n] < 0 and ao[n] != ao[-1] and n != 0:
            if ao[n] < ao[n-1] and ao[n] < ao[n+1]:
                if stock['ticker_id'][0] not in aos:
                    aos.append(stock['ticker_id'][0])
                    print(stock)
print(len(aos))
print(len(tickers))
res = []
for m in aos:
    if m in tickers:
        res.append(m)
print(len(res))
    

                     ticker_id                open               Close  \
date                                                                     
2021-01-20 00:00:00  692976271    17.3799991607666  17.329999923706055   
2021-01-21 00:00:00  692976271  17.299999237060547    17.3799991607666   
2021-01-22 00:00:00  692976271  17.299999237060547                17.5   
2021-01-25 00:00:00  692976271   17.40999984741211   18.59000015258789   
2021-01-26 00:00:00  692976271  18.389999389648438                19.5   
...                        ...                 ...                 ...   
2021-05-03 00:00:00  692976271  18.639999389648438  18.850000381469727   
2021-05-04 00:00:00  692976271  18.299999237060547    18.3799991607666   
2021-05-05 00:00:00  692976271                18.5  18.610000610351562   
2021-05-06 00:00:00  692976271  18.639999389648438  19.290000915527344   
2021-05-07 00:00:00  692976271  19.219999313354492   19.18000030517578   

                                   Hi

                     ticker_id                 open                Close  \
date                                                                       
2021-01-20 00:00:00  517597439  0.32499998807907104  0.32499998807907104   
2021-01-21 00:00:00  517597439  0.32499998807907104  0.28299999237060547   
2021-01-22 00:00:00  517597439  0.28299999237060547   0.3100000023841858   
2021-01-25 00:00:00  517597439  0.38499999046325684  0.27000001072883606   
2021-01-26 00:00:00  517597439  0.33500000834465027   0.3400000035762787   
...                        ...                  ...                  ...   
2021-05-03 00:00:00  517597439   0.3019999861717224  0.29499998688697815   
2021-05-04 00:00:00  517597439  0.29499998688697815  0.29499998688697815   
2021-05-05 00:00:00  517597439  0.29499998688697815  0.29499998688697815   
2021-05-06 00:00:00  517597439  0.29499998688697815  0.29499998688697815   
2021-05-07 00:00:00  517597439  0.29499998688697815  0.29499998688697815   

           

                     ticker_id                open               Close  \
date                                                                     
2021-01-20 00:00:00  197576292   61.91999816894531    62.0099983215332   
2021-01-21 00:00:00  197576292   62.27000045776367    61.7599983215332   
2021-01-22 00:00:00  197576292  61.060001373291016   62.47999954223633   
2021-01-25 00:00:00  197576292    63.0099983215332   61.38999938964844   
2021-01-26 00:00:00  197576292   61.70000076293945  60.290000915527344   
...                        ...                 ...                 ...   
2021-05-03 00:00:00  197576292    61.5099983215332   63.59000015258789   
2021-05-04 00:00:00  197576292   63.18000030517578   64.47000122070312   
2021-05-05 00:00:00  197576292                65.0   64.29000091552734   
2021-05-06 00:00:00  197576292   64.62999725341797   65.18000030517578   
2021-05-07 00:00:00  197576292   64.77999877929688   65.04000091552734   

                                   Hi

14
32
8


In [172]:
# check for trending ma
conn = psycopg2.connect(database="postgres", user=keys.user, password=keys.password, host=keys.host, port="5432")
cur = conn.cursor()
#cur.execute(f"select * from master_ticker_list limit 5;")
print(res)
arg = ""
for d in range(len(res)):
    if res[d] == res[-1]:
        arg += str(res[d])
    else:
        arg += str(res[d]) + ','
print(arg)
cur.execute(f"select * from historical_stock_data where ticker_id in ({arg}) and date >= '2021-01-01';")
data1 = cur.fetchall()
print(data1)
conn.commit()
conn.close()
    

[692976271, 896553772, 820715686, 853155242, 931963840, 527234425, 197576292, 224774719]
692976271,896553772,820715686,853155242,931963840,527234425,197576292,224774719
[(692976271, Decimal('17.600000381469727'), Decimal('17.1200008392334'), Decimal('17.81999969482422'), Decimal('17.010000228881836'), Decimal('2435300.0'), '2021-01-04 00:00:00'), (692976271, Decimal('17.059999465942383'), Decimal('17.049999237060547'), Decimal('17.5'), Decimal('16.979999542236328'), Decimal('1869200.0'), '2021-01-05 00:00:00'), (692976271, Decimal('17.1200008392334'), Decimal('17.360000610351562'), Decimal('17.639999389648438'), Decimal('17.0'), Decimal('3441300.0'), '2021-01-06 00:00:00'), (692976271, Decimal('17.110000610351562'), Decimal('16.850000381469727'), Decimal('17.360000610351562'), Decimal('16.81999969482422'), Decimal('2577000.0'), '2021-01-07 00:00:00'), (692976271, Decimal('16.81999969482422'), Decimal('16.610000610351562'), Decimal('16.979999542236328'), Decimal('16.149999618530273'), D

In [175]:
#now have data, group it back
res_arr = []
for w in res:
    arr = []
    for l in data1:
        if w in l:
            arr.append(l)
        if l == data1[-1]:
            res_arr.append(arr)
            
success = []
print(res_arr)
for tick in res_arr:
    name = []
    _open = []
    close = []
    high = []
    low = []
    volume = []
    index = []
    for b in tick:
        name.append(b[0])
        _open.append(b[1])
        close.append(b[2])
        high.append(b[3])
        low.append(b[4])
        volume.append(b[5])
        index.append(b[6])
    data = {
        'ticker_id': name,
        'open': _open,
        'close': close,
        'high':high,
        'low': low,
        'volume': volume,
        'date': index,
    }
    cols = ['ticker_id', 'open', 'close', 'high', 'low', 'volume', 'date']
    
    df = pd.DataFrame (data, columns = cols)
    df.index = index
    ma = []
    for g in range(len(df['close'])):
        if g > 40:
            avg = sum(df['close'][g-40:g]) / 40
            ma.append(avg)
        else:
            ma.append(0)
    df['ma'] = ma
    counter = 0
    for f in range(len(df[-20:-1])):
        if df['close'][f] > df['ma'][f]:
            counter += 1
            print('pos')
        else:
            counter -= 1
            print('neg')
    
    if counter > 0:
        print(counter, df['ticker_id'][0])
        success.append(df['ticker_id'][0])
    
print(success)
        
    

[[(692976271, 'ACI', 'Albertsons')], [(896553772, 'ACNB', 'ACNB')], [(820715686, 'AAMC', 'Altisource')], [(853155242, 'ABCL', 'AbCellera')], [(931963840, 'ABMC', 'American')], [(527234425, 'ABNB', 'Airbnb')], [(197576292, 'ACA', 'Arcosa')], [(224774719, 'ACAD', 'ACADIA')]]


IndexError: tuple index out of range

In [174]:
conn = psycopg2.connect(database="postgres", user=keys.user, password=keys.password, host=keys.host, port="5432")
cur = conn.cursor()
arg = ""
for d in range(len(success)):
    if success[d] == success[-1]:
        arg += str(success[d])
    else:
        arg += str(success[d])  + ','
print(arg)
cur.execute(f"select * from master_ticker_list where id in ({arg});")
data1 = cur.fetchall()
print(data1)
conn.commit()
conn.close()

692976271,896553772,820715686,853155242,931963840,527234425,197576292,224774719
[(820715686, 'AAMC', 'Altisource'), (853155242, 'ABCL', 'AbCellera'), (931963840, 'ABMC', 'American'), (527234425, 'ABNB', 'Airbnb'), (197576292, 'ACA', 'Arcosa'), (224774719, 'ACAD', 'ACADIA'), (692976271, 'ACI', 'Albertsons'), (896553772, 'ACNB', 'ACNB')]
